# 马尔可夫决策过程 (MDP)
:label:`sec_mdp`
在本节中，我们将讨论如何使用马尔可夫决策过程（MDPs）来构建强化学习问题，并详细描述MDP的各种组成部分。

## MDP的定义

马尔可夫决策过程 (MDP) :cite:`BellmanMDP` 是一个模型，用于描述系统状态随着对系统施加不同动作而演变的方式。几个不同的量共同构成了一个MDP。

![一个简单的网格世界导航任务，机器人不仅需要找到通往目标位置（显示为绿色房屋）的路径，还需要避开陷阱位置（显示为红色十字标志）。](../img/mdp.png)
:width:`250px`
:label:`fig_mdp`

* 设$\mathcal{S}$为MDP中的状态集合。以:numref:`fig_mdp`所示的具体例子来看，对于正在导航网格世界的机器人而言，$\mathcal{S}$对应于机器人在任何给定时间步可以位于的位置集合。
* 设$\mathcal{A}$为机器人在每个状态下可以采取的动作集合，例如“前进”、“向右转”、“向左转”、“保持在同一位置”等。动作可以将机器人的当前状态改变为集合$\mathcal{S}$内的其他状态。
* 可能的情况是我们并不确切知道机器人是如何移动的，只知道其近似情况。我们在强化学习中这样建模：如果机器人执行了“前进”的动作，可能会有很小的概率它会停留在当前位置，另一个小概率是它“向左转”，等等。数学上，这相当于定义了一个“转移函数”$T: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \to [0,1]$，使得$T(s, a, s') = P(s' \mid s, a)$利用条件概率表示从状态$s$出发执行动作$a$到达状态$s'$的概率。转移函数是一个概率分布，因此我们有$\sum_{s' \in \mathcal{S}} T(s, a, s') = 1$适用于所有$s \in \mathcal{S}$和$a \in \mathcal{A}$，即如果机器人采取了某个动作，它必须转移到某个状态。
* 我们现在通过“奖励”$r: \mathcal{S} \times \mathcal{A} \to \mathbb{R}$的概念来构造哪些动作是有用的哪些不是。如果机器人在状态$s$下采取了动作$a$，则说机器人获得了奖励$r(s,a)$。如果奖励$r(s, a)$很大，这表明在状态$s$下采取动作$a$更有利于实现机器人的目标，即前往绿色房子。如果奖励$r(s, a)$很小，则动作$a$对该目标的贡献较小。重要的是要注意，奖励是由用户（创建强化学习算法的人）根据目标设计的。

## 回报与折扣因子

上述的不同部分共同构成了一种马尔可夫决策过程 (MDP)
$$\textrm{MDP}: (\mathcal{S}, \mathcal{A}, T, r).$$

让我们考虑当机器人从特定状态$s_0 \in \mathcal{S}$开始并继续采取行动从而形成轨迹时的情形
$$\tau = (s_0, a_0, r_0, s_1, a_1, r_1, s_2, a_2, r_2, \ldots).$$

在每个时间步$t$，机器人处于状态$s_t$并采取动作$a_t$，这导致了奖励$r_t = r(s_t, a_t)$。轨迹的*回报*是指沿着该轨迹机器人获得的总奖励
$$R(\tau) = r_0 + r_1 + r_2 + \cdots.$$

强化学习的目标是找到具有最大*回报*的轨迹。

想象一下机器人继续在网格世界中行进却永远无法达到目标位置的情况。在这种情况下，轨迹中的状态和动作序列可能是无限长的，任何这种无限长轨迹的*回报*也将是无限的。为了即使对于这样的轨迹也能使强化学习公式有意义，我们引入了折扣因子$\gamma < 1$的概念。我们把打折后的*回报*写作
$$R(\tau) = r_0 + \gamma r_1 + \gamma^2 r_2 + \cdots = \sum_{t=0}^\infty \gamma^t r_t.$$

注意，如果$\gamma$非常小，那么机器人在未来很远的地方，比如说$t = 1000$时所获得的奖励将被$\gamma^{1000}$这个因素严重打折。这鼓励机器人选择能够快速达成其目标的较短轨迹，即前往网格世界示例中的绿色房子（见:numref:`fig_mdp`）。对于较大的折扣因子值，比如$\gamma = 0.99$，机器人被鼓励去*探索*然后找到到达目标位置的最佳轨迹。

## 马尔可夫假设的讨论

让我们考虑一个新的机器人，其中的状态$s_t$如上所述代表位置，但动作$a_t$是机器人对其轮子应用的加速度而不是像“前进”这样的抽象命令。如果这个机器人在状态$s_t$时有一些非零速度，那么下一个位置$s_{t+1}$将是过去位置$s_t$、加速度$a_t$以及机器人在时间$t$的速度（与$s_t - s_{t-1}$成比例）的一个函数。这表明我们应该有

$$s_{t+1} = \textrm{某种函数}(s_t, a_t, s_{t-1});$$

在我们的情况下，“某种函数”将是牛顿运动定律。这与仅仅依赖于$s_t$和$a_t$的转移函数大不相同。

马尔可夫系统是指下一个状态$s_{t+1}$仅是当前状态$s_t$和在当前状态下采取的动作$a_t$的函数的所有系统。在马尔可夫系统中，下一个状态不依赖于过去采取的动作或机器人过去所处的状态。例如，上面以加速度作为动作的新机器人就不是马尔可夫式的，因为下一个位置$s_{t+1}$取决于通过速度关联的先前状态$s_{t-1}$。虽然看起来系统的马尔可夫性质是一种限制性假设，但实际上并非如此。马尔可夫决策过程仍然能够模拟一大类现实系统。例如，对于我们的新机器人，如果我们选择状态$s_t$为元组$(\textrm{位置}, \textrm{速度})$，那么系统就是马尔可夫式的，因为它的下一个状态$(\textrm{位置}_{t+1}, \textrm{速度}_{t+1})$只依赖于当前状态$(\textrm{位置}_t, \textrm{速度}_t)$和当前状态下的动作$a_t$。

## 总结
强化学习问题通常使用马尔可夫决策过程进行建模。一个马尔可夫决策过程(MDP)由四个实体的元组定义$(\mathcal{S}, \mathcal{A}, T, r)$，其中$\mathcal{S}$是状态空间，$\mathcal{A}$是动作空间，$T$是编码MDP转移概率的转移函数，而$r$是在特定状态下采取动作所获得的即时奖励。

## 练习

1. 假设我们要设计一个MDP来建模[MountainCar](https://www.gymlibrary.dev/environments/classic_control/mountain_car/)问题。
    1. 状态集会是什么？
    2. 动作集会是什么？
    3. 可能的奖励函数有哪些？
2. 你将如何为类似[Pong游戏](https://www.gymlibrary.dev/environments/atari/pong/)这样的Atari游戏设计一个MDP？

[讨论](https://discuss.d2l.ai/t/12084)